In [ ]:
# look at results
import ase.io.gaussian
from ase.calculators.calculator import PropertyNotImplementedError

import rmgpy.species
import rmgpy.chemkin

import glob
import numpy as np


import rmgpy.species
import rmgpy.chemkin
import autotst.species
from autotst.calculator.gaussian import Gaussian
import autotst.calculator.statmech

import matplotlib.pyplot as plt

In [ ]:
my_files = glob.glob('nheptane/*.log')
my_files.sort()
energies = np.zeros(len(my_files))
for i, filename in enumerate(my_files):
    with open(filename, 'r') as f:
        try:
            atoms = ase.io.gaussian.read_gaussian_out(f)
            energy = atoms.get_potential_energy()
            energies[i] = energy
            print(filename, energy)
        except IndexError:
            print(f'Index Error: {filename}')
        except PropertyNotImplementedError:
            print(f'Missing: {filename}')

In [ ]:
f = open('nheptane/cf7.log', 'r')
my_obj = ase.io.gaussian.read_gaussian_out(f)
print(my_obj.get_potential_energy())

In [ ]:
lowest = list(energies).index(np.min(energies))
print(my_files[lowest])

In [ ]:
%matplotlib inline
plt.scatter(range(0, len(energies)), energies, marker='*')
ax = plt.gca()
ax.set_ylim([1.0001 * np.min(energies), -7519.2])
plt.show()

In [ ]:
# make a conformer object again
# load
lowest_cf_file = 'nheptane/cf39.log'
with open(lowest_cf_file, 'r') as f:
    atoms = ase.io.gaussian.read_gaussian_out(f)
new_cf = autotst.species.Conformer(smiles='CCCCCCC')
new_cf._ase_molecule = atoms
new_cf.update_coords_from(mol_type="ase")
new_cf.view()

In [ ]:
gaussian = Gaussian(conformer=new_cf)
calc = gaussian.get_rotor_calc()
calc.label = f'rotor1'
# calc.label = f'rotor{i:04}'

In [ ]:
calc.write_input(new_cf.ase_molecule)

In [ ]:
dir(atoms)

In [ ]:
atoms.get_potential_energy()
# atoms.get_dipole_moment()

In [ ]:
new_cf.get_torsions()

In [ ]:
rotor_stats = autotst.calculator.statmech.StatMech('fake_rxn')
rotor_stats.get_rotor_info(new_cf, new_cf.get_torsions()[0])

In [ ]:
def get_rotor_info(conformer, torsion, tor_log):
    _, j, k, _ = torsion.atom_indices

    # Adjusted since mol's IDs start from 0 while Arkane's start from 1
    tor_center_adj = [j+1, k+1]


    top_IDs = []
    for num, tf in enumerate(torsion.mask):
        if tf:
            top_IDs.append(num)

    # Adjusted to start from 1 instead of 0
    top_IDs_adj = [ID+1 for ID in top_IDs]

    info = f"     HinderedRotor(scanLog=Log('{tor_log}'), pivots={tor_center_adj}, top={top_IDs_adj}, fit='fourier'),"

    return info

In [ ]:
tor0_log = '/home/harris.se/rmg/rmg_tools/autotst/thermo/first_rotor/rotor0.log'
for tor in new_cf.get_torsions():
    print(get_rotor_info(new_cf, tor, tor0_log))

# tor0_log = '/home/harris.se/rmg/rmg_tools/autotst/thermo/first_rotor/rotor0.log'
# get_rotor_info(new_cf, new_cf.get_torsions()[0], tor0_log)

In [ ]:
# load the model
chemkin_path = "/home/harris.se/rmg/rmg_tools/uncertainty/nheptane/chem_annotated.inp"
dictionary_path = "/home/harris.se/rmg/rmg_tools/uncertainty/nheptane/species_dictionary.txt"
transport_path = "/home/harris.se/rmg/rmg_tools/uncertainty/nheptane/tran.dat"
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(
    chemkin_path,
    dictionary_path=dictionary_path,
    transport_path=transport_path
)
print (f"Loaded model with {len(species_list)} species and {len(reaction_list)} reactions")

In [ ]:
def get_species_index(list_of_species, name):
    for i, sp in enumerate(list_of_species):
        if str(sp) == name:
            return i
    return -1

In [ ]:
nheptane_rmg = species_list[get_species_index(species_list, 'nheptane(1)')]
nheptane = autotst.species.Species([nheptane_rmg.smiles])

In [ ]:
nheptane.conformers['CCCCCCC'] = [new_cf]

In [ ]:
stat = autotst.calculator.statmech.StatMech(0)
stat.write_species_files(nheptane)

In [ ]:
nheptane.conformers

In [ ]:
stat.write_conformer_file(new_cf)

In [ ]:
stat.directory